# Distribuições Territoriais

Esta seção constrói mapas básicos das áreas de estudo: as cidades de Belo Horizonte, Rio de Janeiro e São Paulo. Especificamente, serão consolidadas e apresentadas as distribuições territoriais de variáveis demográficas e socioeconômicas de interesse. Adicionalmente, há o complemento de atributos territoriais que entende-se que são, também, variáveis explicativas. Especificamente, o foco estará, preliminarmente, nas seguintes variáveis, as quais serão elaboradas em maior profundidade mais à frente.

- Contagens populacionais
- Rendimentos médios
- Contagens de postos de trabalho
- Índice de bem estar urbano
- Medidas geométricas e topológicas da malha de vias caminháveis

Os itens que seguem tratam de detalhar os passos metodológicos utilizados, o embasamento teórico-conceitual (ou empírico) que é subjacente a esses passos, bem como as fontes de dados. Ainda, logo de início estão listadas as bibliotecas em Python que dão suporte a todo o processo. Maiores detalhes a respeito do funcionamento dessas bibliotecas são dados na medida do necessário. 

Os dados consolidados serão apresentados em formato de mapas (estáticos) e salvos para uso nas etapas posteriores do artigo. Análises espaciais e estat;isticas mais aprofundadas serão conteúdo de scripts mais à frente. De particular interesse será verificar a articulação entre as porções do território de cada cidade, uma vez que esses fluxos sejam confrontados com os fluxos e as infraestruturas de transporte motorizado.

# Procedimentos Preliminares

## Bibliotecas

In [1]:
import os
import pathlib
import re
import requests
import subprocess
from string import ascii_uppercase
from requests.exceptions import ConnectTimeout

#import googlemaps
#import h3
import basedosdados as bd
import geopandas as gpd
import IPython.display as display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from geobr import read_metro_area, read_municipality, read_census_tract, lookup_muni
from getpass import getpass
from shapely.geometry import Point, LineString, box
from tobler.area_weighted import area_interpolate
from tobler.util import h3fy

ModuleNotFoundError: No module named 'basedosdados'

## Parâmetros de Base

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('Solarize_Light2')

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
db = pathlib.Path(
    os.environ.get('DB_FOLDER')
    )

out_folder = os.environ.get('OUT_FOLDER')
out_folder = pathlib.Path(out_folder)
out_folder = out_folder / 'A'

In [ ]:
gcloud_id = getpass('Google Cloud Services billing ID:')

In [ ]:
# SIRGAS 2000 / zona UTM 23S
EPSG = 31983

# IBGE codes for selected municipalities⨥
GEOCODE = {
    'belo_horizonte': 3106200,
    }

In [ ]:
muni_dict = {str(v): k.replace('_', ' ').title() for k, v in GEOCODE.items() }

⨥ ver https://www.ibge.gov.br/explica/codigos-dos-municipios.php para a Tabela de Códigos de Municípios do IBGE

# Das Malhas Territorias

A malha hexagonal H3 será utilizada como recorte espacial principal ao longo das análises†. Recortes hexagonais apresentam vantagens importantes frente a outras geometrias, sobretudo, eles podem reduzir vieses amostrais derivados de efeitos de borda e, adicionalmente, são mais mais adequados do que malhas retangulares para analisar fenômenos em que relações de vizinhança e conectividade são importantes (Birch, Oom, & Beecham, 2007). Essas vantagens da malha hexagonal serão mais imporantes quando estiverem sendo executadas as análises de acessibilidade, mas, até lá, também foi mostrado por Birch et al. (2007) que hexágonos favorecem a visualização de padrões e mitigam, em alguma medida, efeitos de borda.

Esses efeitos de borda, simplificadamente, dizem respeito a erros de representação na variação local dos dados. Isto é, outras geometrias — malhas quadriculadas, p. ex. — são mais propensas a apresentar variações bruscas e artificiais entre vizinhos — uma quadrícula com valores muito altos ladeada por quadrículas com valores muito baixos. Isso prejudica a análise e a visualização, assim favorecendo erros de interpretação.

Dados socioeconômicos e demográficos, por sua vez, serão imputados em cada hexágono a partir do enso demográfico de 2010. Esses dados censitários serão requisitados ao datalake público da Base dos Dados‡, enquanto que as malhas desses setores serão obtidas junto à biblioteca geobr⨥. Assim, de posse das malhas do IBGE e dos respectivos dados, procedimentos de interpolação espacial⁙ farão a transferência dos dados do censo para os hexágonos.

Complementarmente, para além das informações dos setores censitários, a grade estatística⁜ também será utilizada como insumo, a qual apresenta contagens populacionais com uma resolução maior que a dos agregados por setor: há uma malha com resolução de 200x200 m em áreas urbanas e de 1x1 km em áreas rurais. A grade é importante porque remove as áreas inabitadas — a Floresta d Tijuca, no Rio de Janeiro, p. ex. — e porque permitem localizar o centroide dos hexágonos de maneira mais precisa: seguindo recomendação de Stępniak and Jacobs-Crisioni (2017), o centro de cada geometria deve ser ponderado pela distribuição populacional nela contida, a fim de que haja uma representação mais precisa dos tempos de viagem entre geometrias.

Resta definir qual a resolução dessas malha. Conforme demonstrado, por exemplo, em Pereira et al. (2019), é importante avaliar distintos tamanhos de geometria, a fim de verificar se os resultados obtidos são realmente robustos ou se não passam de artefatos computacionais. Mas, quais resoluções⁕? A menor escala na qual ainda há dados disponíveis é a grade estatística, de maneira que hexágonos menores que ela não agregariam nenhum valor à análise, ao mesmo tempo em que elevariam, desnecessariamente, o tempo de processamento computacional. Com efeito, a menor resolução utilizada é a 9, com área relativamente próxima a um quadrado de lado 200m. No outro extremo, resta definir a maior resolução utilizada. A maior delas será a 7, com área de aproximadamente 5 km2. Tamahos maiores agregariam grandes regiões muito distintas entre si, contribuindo em demasiado para uma ilusão de homogeneidade a qual não existe — p. ex., a resolução 6 já conta com área de cercade 35km2. Portanto, serão avaliadas as resoluções 7, 8 e 9.

    Birch, C. P., Oom, S. P., & Beecham, J. A. (2007). Rectangular and hexagonal grids used for observation, experiment and simulation in ecology. Ecological modelling, 206(3-4), 347-359.
    
    Pereira, R. H., Banister, D., Schwanen, T., & Wessel, N. (2019). Distributional effects of transport policies on inequalities in access to opportunities in Rio de Janeiro. Journal of Transport and Land Use, 12(1), 741-764.
    
    Stępniak, M., & Jacobs-Crisioni, C. (2017). Reducing the uncertainty induced by spatial aggregation in accessibility and spatial interaction applications. Journal of Transport Geography, 61, 17-29.

    † Especificamente, será feito uso das bibliotecas em Python implementadas pela Uber (https://h3geo.org/), baseadas no sistema proposto em:

        Sahr, K., White, D., & Kimerling, A. J. (2003). Geodesic discrete global grid systems. Cartography and Geographic Information Science, 30(2), 121-134.
        
    ‡ ver https://basedosdados.org/

    ⨥ ver https://github.com/ipeaGIT/geobr
    
    ⁙ https://github.com/pysal/tobler

    ⁜ obtida diretamente do site do IBGE em https://censo2010.ibge.gov.br/component/saladeimprensa/?busca=1&id=3&idnoticia=3123&t=grade-estatistica-permite-obter-dados-censo-2010-diversos-recortes-espaciais&view=noticia
    
    ⁕ ver https://h3geo.org/docs/core-library/restable para uma lista de resoluções e as respectivas dimensões

## Malhas Censitárias

In [ ]:
def _get_city_tracts(ibgeID, out_crs):
    return (
        read_census_tract(ibgeID, year=2010).to_crs(epsg=out_crs)
                                            .astype({
                                                'code_muni': int,
                                                })
                                            .drop(columns=[
                                                'code_neighborhood',
                                                'code_subdistrict',
                                                'code_district',
                                                'code_state',
                                                ])
        )


def get_tracts(geocodes, out_crs=31983):
    tracts = [
        _get_city_tracts(ibgeID, out_crs)
        for ibgeID
        in geocodes.values()
        ]
    
    return pd.concat(tracts, ignore_index=True)
        

In [ ]:
tracts = get_tracts(GEOCODE)

## Hexágonos H3

In [ ]:
def _get_hexagons(muni_tracts, r, out_crs):
    hexes = h3fy(muni_tracts.buffer(0), resolution=r).to_crs(epsg=out_crs)
    hexes['aperture'] = r
    
    return hexes


def _hexagons_by_resolution(muni_tracts, out_crs):
    hexes = pd.concat([
        _get_hexagons(muni_tracts, r, out_crs)
        for r
        in [9, 10, 11, 12]
        ])
    
    hexes['name_muni'] = muni_tracts.name_muni.iloc[0]
    hexes['code_muni'] = muni_tracts.code_muni.iloc[0].astype(int)
    
    return (
        hexes.reset_index()
             .set_index([
                 'name_muni',
                 'code_muni',
                 'aperture',
                 'hex_id',
                 ])
        )


def _get_border(name):
    code = lookup_muni(name).code_muni.squeeze()
    return read_municipality(code).to_crs(31983)


def hexagons_by_municipality(tracts, out_crs=31983):
    return pd.concat([
        _hexagons_by_resolution(_get_border(name_muni), out_crs)
        for name_muni, _
        in tracts.groupby('name_muni')
        ])

In [ ]:
hexes = hexagons_by_municipality(tracts)

## Grades Estatísticas

As grades estatísticas dependem de procedimento particular. Devido à sua alta resolução, elas são compartimentadas em setores pelo IBGE, a fim de otimizar o uso e gerenciamento do espaço computacional,conforme mapa abaixo. Desse mapa é possível verificar que Belo Horizonte pertence ao ID_36:

Cabe, por fim, reforçar algo que já evidente no mapa: os setores contém muito mais do qu as cidades analisadas. Com efeito, após importá-los todos, serão descartadas as partes que não fazem interseção com os territórios municipais.

In [ ]:
display.Image('https://wiki.openstreetmap.org/w/images/7/74/Articulacao.jpg')

In [ ]:
# Helper functions to read and parse statistical grid

def _fix_zipped_path_to_shapes(inpath):
    try:
        return 'zip://' + inpath.as_posix()
    except AttributeError:
        return 'zip://' + inpath


def _reindex_grid(grid):
    d = {
        'ID_UNICO': 'cell_id',
        'POP': 'pop_total',
        'geometry': 'geometry'
        }
    return (
        grid.reindex(columns=d)
            .rename(columns=d)
            .set_index('cell_id')
        )


def _get_municipality_grid(path,
                           muni_name,
                           out_crs,
                           is_zipped):
    if is_zipped:
        inpath = _fix_zipped_path_to_shapes(path)
    
    gdf = (
        gpd.read_file(path)
           .to_crs(epsg=out_crs)
           .pipe(_reindex_grid)
        )
    
    gdf['name_muni'] = muni_name
    
    return gdf


def grid_by_municipality(inpaths,
                         out_crs,
                         is_zipped=True):
    return pd.concat([
        _get_municipality_grid(path,
                               muni_name,
                               out_crs,
                               is_zipped)
        for muni_name, path
        in inpaths.items()
        ])
    

In [ ]:
# Reads statistical grid

grid_files = {
    'belo_horizonte': 'grade_estatistica_id36.zip',
    }
inpaths = {
    k: db / 'census' / 'grade_estatistica' / v
    for k, v
    in grid_files.items()
    }
grid = grid_by_municipality(inpaths,
                            EPSG,
                           )

In [ ]:
def filter_study_areas(grid, mask):
    return (
        grid.pipe(
                gpd.sjoin,
                mask.reset_index(),
                predicate='intersects',
                )
            .reset_index()
            .drop_duplicates(subset='cell_id')
            .drop(columns='index_right')
            .set_index('cell_id')
        
        )

In [ ]:
grid = filter_study_areas(
    grid,
    hexes.xs(key=10, level='aperture')
    )

# A Leitura dos Atributos Territoriais

Resgatando o que foi dito no preâmbulo deste script, estamos interessados nas seguintes variáveis:
1. Contagens populacionais
2. Contagem de domicílios
3. Rendimentos médios
4. Contagens de postos de trabalho
5. Índice de bem estar urbano

A ideia, portanto, é fazer a leitura e o ajuste dessas variáveis para que possam ser imptuadas nas malhas hexagonais H3.

## Rendimentos Médios

Os rendimentos serão obtidos a partir da API da Base dos Dados. Especificamente, utiliza-se a seguinte variável da tabela básica do Censo de 2010:

- v009: Valor do rendimento nominal médio mensal das pessoas de 10 anos ou mais de idade (com e sem rendimento)

In [ ]:
def _sql_style_list(sequence):
    single_quoted_elements = [f"'{c}'" for c in sequence]
    return ', '.join(single_quoted_elements)

In [ ]:
def read_census_data(query, gcloud_id):
    data = bd.read_sql(query,
                       billing_project_id=gcloud_id)
    
    return (
        data.astype({'id_setor_censitario': np.int64})
            .astype({'id_setor_censitario': str})
            .rename(
                columns={
                    'v009': 'rendimento_medio',
                    'v002': 'habitantes',
                    'v001': 'domicilios',
                    }
                )
        )

In [ ]:
query = (
    'SELECT id_setor_censitario, v009, v002, v001 '
    'FROM basedosdados.br_ibge_censo_demografico.setor_censitario_basico_2010 '
    f"WHERE id_setor_censitario IN ({_sql_style_list(tracts.code_tract.unique())})"
    )
tract_data = read_census_data(query, gcloud_id)

In [ ]:
tract_data.head(3)

## Índice de Bem-Estar Urbano

Esse é um indicador desenvolvido pelo INCT Observatório das Metrópoles. A métrica 

    procura avaliar a dimensão urbana do bem-estar usufruído pelos cidadãos brasileiros promovido pelo mercado, via o consumo mercantil, e pelos serviços sociais prestados pelo Estado. Tal dimensão está relacionada com as condições coletivas de vida promovidas pelo ambiente construído da cidade, nas escalas da habitação e da sua vizinhança próxima, e pelos equipamentos e serviços urbanos. (Ribeiro & Ribeiro, 2016).

Há cinco dimensões: Mobilidade Urbana, Condições Ambientais Urbanas, Condições Habitacionais Urbanas, Atendimento de Serviços Coletivos Urbanos e Infraestrutura Urbana. Maior detalhamento a respeito das de como é calculada cada uma dessas dimensões podem ser encontrados diretamente em Ribeiro e Ribeiro (2016). Aqui, a exposição se limita a explicar quais dessas dimensões serão utilizadas e por quê.

A dimensão da mobilidade não será utilizada. Essa dimensão tem como principal pilar o tempo de viagem casa-trabalho, tal como capturado pelo Censo de 2010. A variável foca apenas no tempo e, para análise à qual este artigo se propõe, seria necessário um conhecimento mais profundo que envolve, dentre outras, variáveis tais quais (a) pares de origem e destino, (b) modos de viagem — se coletivos ou individuais, p. ex. — e (c) gradientes de acessibilidade ao longo do território. Portanto, essa análise mais minuciosa precisa ser desenvolvida de outra forma, o que será feito em scripts posteriores a partir de pesquisas de origem e destino, dados de GTFS etc.

Outra dimensão que não será utilizada é a de condições habitacionais, pois diz respeito à qualidade física dos domicílios de cada setor e à densidade ocupacional no interior de cada residência. É razoável supor que isso faça pouca diferença para o estabelecimento de empresas ou serviços. Por um lado, locais com baixa condição habitacional podem ser menos atrativos porque tendem a estar em áreas mais periféricas e de menor qualidade urbanística em geral. Por outro, há outras dimensões do IBEU que tratam diretamente da qualidade do espaço público e da qualidade das infraestruturas urbanas existentes. Deve-se ter em mente o seguinte: a hipótese central do artigo é que o estabelecimento de centros comerciais depende do acesso à massa salarial local, acesso esse mediado pelo sistema de transportes. Nesse sentido, pouco importa o que acontece "intramuros", importa o mercado ao qual se tem acesso. Ao cabo, essa variável pode ser vista por dois ângulos: ou como redundante ou como irrelevante.

As demais dimensões serão utilizadas pois elas são entendidas como essenciais à análise em questão. Rememorando, são elas as dimensões (a) de serviços coletivos urbanos, (b) de condições ambientais urbanas e (c) de infraestrutura urbana. A essa altura, já é evidente que o que aqui será produzido é uma versão parcial do IBEU, cujo foco está na forma urbana construída.

Em certa medida,é possível afirmar que também haveria uma correlação importante entre a qualidade do ambiente urbano e os rendimentos do entorno. O que, por um lado, pode levantar suspeitas a respeito da redundância dessa variável. Contudo, entende-se que esse pode ser um critério de desempate interessante, em casos de locais com acesso semelhante. A pertinência dessa variável será demonstrada — ou não — com as análises do modelo.

A respeito da dimensão de serviços coletivos urbanos, Ribeiro e Ribeiro (2016, p. 4) alegam que "esses são indicadores que expressam os serviços públicos essenciais para garantia de bem-estar urbano, independentemente de ser _ofertado por empresas públicas ou por empresas privadas através de concessão pública_ [ênfase adicionada]". Disso, subentende-se que um atendimento adequado estaria refletido pelas seguintes variáveis extraídas da tabela de características gerais dos domicílios:

- v002 : Domicílios particulares e domicílios coletivos

- IBEU Serviços Coletivos Urbanos
    - v012 : Domicílios particulares permanentes com abastecimento de água da rede geral
    - v017 : Domicílios particulares permanentes com banheiro de uso exclusivo dos moradores ou sanitário e esgotamento sanitário via rede geral de esgoto ou pluvial
    - v035 : Domicílios particulares permanentes com lixo coletado
    - v044 : Domicílios particulares permanentes com energia elétrica de companhia distribuidora

As outras duas dimensões são obtidas a partir da tabela com informações a respeito do entorno das quadras dos setores censitários. As variáveis são as seguintes:

- v001 : Domicílios particulares permanentes

- IBEU Infraestrutura Urbana
    - v002 : Domicílios particulares permanentes próprios – Existe identificação do logradouro
    - v004 : Domicílios particulares permanentes alugados – Existe identificação do logradouro
    - v006 : Domicílios particulares permanentes cedidos – Existe identificação do logradouro
    - v008 : Domicílios particulares permanentes próprios – Existe iluminação pública
    - v010 : Domicílios particulares permanentes alugados – Existe iluminação pública
    - v012 : Domicílios particulares permanentes cedidos – Existe iluminação pública
    - v014 : Domicílios particulares permanentes próprios – Existe pavimentação
    - v016 : Domicílios particulares permanentes alugados – Existe pavimentação
    - v018 : Domicílios particulares permanentes cedidos – Existe pavimentação
    - v020 : Domicílios particulares permanentes próprios – Existe calçada
    - v022 : Domicílios particulares permanentes alugados – Existe calçada
    - v024 : Domicílios particulares permanentes cedidos – Existe calçada
    - v026 : Domicílios particulares permanentes próprios – Existe meio-fio/guia
    - v028 : Domicílios particulares permanentes alugados – Existe meio-fio/guia
    - v030 : Domicílios particulares permanentes cedidos – Existe meio-fio/guia
    - v032 : Domicílios particulares permanentes próprios – Existe bueiro/boca-de-lobo
    - v034 : Domicílios particulares permanentes alugados – Existe bueiro/boca-de-lobo
    - v036 : Domicílios particulares permanentes cedidos – Existe bueiro/boca-de-lobo
    - v038 : Domicílios particulares permanentes próprios – Existe rampa para cadeirante
    - v040 : Domicílios particulares permanentes alugados – Existe rampa para cadeirante
    - v042 : Domicílios particulares permanentes cedidos – Existe rampa para cadeirante
        

- IBEU Condições Ambientais Urbanas
    - v044 : Domicílios particulares permanentes próprios – Existe arborização
    - v046 : Domicílios particulares permanentes alugados – Existe arborização
    - v048 : Domicílios particulares permanentes cedidos – Existe arborização
    - v051 : Domicílios particulares permanentes próprios – Não existe esgoto a céu aberto
    - v053 : Domicílios particulares permanentes alugados – Não existe esgoto a céu aberto
    - v055 : Domicílios particulares permanentes cedidos – Não existe esgoto a céu aberto
    - v057 : Domicílios particulares permanentes próprios – Não existe lixo acumulado nos logradouros
    - v059 : Domicílios particulares permanentes alugados – Não existe lixo acumulado nos logradouros
    - v061 : Domicílios particulares permanentes cedidos – Não existe lixo acumulado nos logradouros

Listadas todas as variáveis necessárias, restam algumas notas metodológicas. A cada variável está associado um indicador que diz respeito à proporção de domicílios de cada setor censitário que tem o atributo sendo medido. Por exemplo, se o indicador referente à arborização tiver valor de 0.72, isso significa que 72% dos domicílios daquele setor possuem arborização em seu entorno. Em seguida, o restantante das contas envolve médias aritméticas simples: cada dimensão é representada pela média de seus indicadores constituintes, enquanto  que o IBEU é a média de cada dimensão. Cabe ressaltar outra diferença em relação ao IBEU original: aqui utilizamos como base de cálculo as proporções de domicílios, em vez da proporção de _pessoas_.

    
        Ribeiro, L. C. Q., Ribeiro, M. G. (2016). IBEU-Municipal: Índice de Bem-Estar Urbano dos Municípios Brasileiros. Letra Capital. https://www.observatoriodasmetropoles.net.br/wp-content/uploads/2020/08/ibeumunicipal_2016.pdf

### IBEU Serviços Coletivos Urbanos

In [ ]:
urban_services_vars = [
    'id_setor_censitario',
    'v002',
    'v012', 
    'v017',
    'v035',
    'v044',
    ]

In [ ]:
query = (
    f"SELECT {', '.join(urban_services_vars)} "
    "FROM basedosdados.br_ibge_censo_demografico.setor_censitario_domicilio_caracteristicas_gerais_2010 "
    f"WHERE id_setor_censitario IN ({_sql_style_list(tracts.code_tract.unique())});"
    )

In [ ]:
services = bd.read_sql(
    query,
    billing_project_id=gcloud_id,
    )

In [ ]:
services.head(3)

### IBEU Infraestrutura Urbana | Condições Ambientais

In [ ]:
surroundings_vars = (
    ['id_setor_censitario', 'v001']
    + [f'v{v:03}' for v in range(2, 49, 2)]
    + [f'v{v:03}' for v in range(51, 62, 2)]
    )

In [ ]:
query = (
    f"SELECT {', '.join(surroundings_vars)} "
    "FROM basedosdados.br_ibge_censo_demografico.setor_censitario_entorno_2010 "
    f"WHERE id_setor_censitario IN ({_sql_style_list(tracts.code_tract.unique())});"
    )

In [ ]:
surroundings = bd.read_sql(
    query,
    billing_project_id=gcloud_id,
    )

In [ ]:
surroundings.head(3)

Os dados dos atributos territoriais fazem uma desagregação por tipo de domicilio. Para o IBEU, essa subdivisão não existe: o que importaa são as residências dotadas de infraestrutura, indpendentemente de serem alugados, cedidos etc. Com efeito, eles serão agrupados, irrespectivamente da situação de posse.

In [ ]:
def _grouping_windows():
    indices = [idx for idx in range(1, surroundings.shape[1]+1, 3)]
    return [(u, v) for u, v in zip(indices[:-1], indices[1:])]


def _wrangle_df(df, u, v):
    sliced_df = df.iloc[:, u:v].copy()
    name = '-'.join(sliced_df.columns)
    return sliced_df.sum(1).rename(name)


def group_surroundings(surroundings):
    windows = _grouping_windows()
    df = surroundings.set_index('id_setor_censitario')
    grouped_df = [
        _wrangle_df(df, u, v)
        for i, (u, v)
        in enumerate(windows)
        ]
    return (
        pd.concat(grouped_df, axis='columns')
          .merge(
              df.v001,
              left_index=True,
              right_index=True,
              ) 
          .reset_index()
        )

In [ ]:
surroundings = group_surroundings(surroundings)

In [ ]:
surroundings.head()

### IBEU

In [ ]:
# Helper functions to compute IBEU

def _urban_area(df, ref_col):
    """Takes only the portion of the dataset
    the is strictly subject to urban dynamics,
    i.e. excludes census tracts of special areas
    such as university campi or penitentiaries.
    
    Parameters
    ----------
    df : DataFrame
        Attributes by census tract
    ref_col : str
        Column containing variable that is either
        zero or null if census tract is a special area
        
    Returns
    -------
        DataFrame
    """
    mask = (df[ref_col] != 0) & df[ref_col].notnull()
    return df.loc[mask]


def _partial_ibeu(partial_data, ref_col):
    df = _urban_area(partial_data, ref_col)
    return (
        df.set_index('id_setor_censitario')
          .pipe(
              lambda df: df.div(df[ref_col], axis='index')
              )
          .drop(columns=ref_col)
        )
    


def compute_ibeu(surroundings, services):
    return (
        pd.merge(
            _partial_ibeu(surroundings, 'v001'),
            _partial_ibeu(services, 'v002'),
            left_index=True,
            right_index=True,
            )
          .apply(lambda row: np.mean(row), axis='columns')
          .rename('ibeu')
        )


In [ ]:
ibeu = compute_ibeu(surroundings, services)

## Relação Anual de Informações Sociais (RAIS)

A Relação Anual de Informações Sociais (RAIS), do Ministério do Trabalho e Emprego, é uma bas de dados que contém os quantitativos de empregos por estabelecimento, os quais ainda são geolocalizados por CEP. Dessa forma, é possível evidenciar a concentração de empresas e serviços no interior de cada hexágono H3 no interior da área de estudo. Com efeito, é a partir da RAIS que será consolidada a variável dependente do modelo.

Uma limitação dos dados da RAIS é que eles apenas contemplam os empregos formais, assim sub-representando a economia subterrânea, a informalidade. Contudo, essa não chega a ser uma limitação muito importante no contexto deste relatório porque estudos anteriores (Pereira, Banister, et al., 2019) detectaram, a partir da pesquisa de origem e destino de 2012, que há expressiva correlação entre a localização dos empregos formais e informais. Essa correlação foi feita para o caso do Rio de Janeiro, mas é razoável supor que também se repita em São Paulo e Belo Horizonte.

Serão utilizados os anos da RAIS com maior proximidade temporal aos dados disponíveis de mobilidade e do sistema de transportes. No caso de São Paulo e Belo Horizonte, em ambos as últimas pesquisas de origem e destino são de 2012, ao mesmo tempo em que há dados de GTFS do ano de 2014 para ambos os municípios. De fato, de Belo Horizonte, há disponibilidade de uma série histórica anual de dados GTFS, coletados junto aos portais da transparência e junto à empresa de planejamento de transportes local. O Rio de Janeiro, portanto, foi o fator limitante, pois houve dificuldades na obtenção de dados abrangentes que contemplassem todos os modos de transporte municipais. A resolução veio quando a Moovit gentilmente cedeu informações respectivas ao ano de 2014, para todos os modos.

Já para o caso de São Paulo, será utilizada a RAIS do ano de 2017, pois esse é o ano da pesquisa de origm e destino mais recente e, ainda, conseguimos obter dados GTFS de 2017, junto ao Serviço de Informações ao Cidadão (SIC), o qual recebe e registra pedidos de acesso à informação feitos por cidadãos e cidadãs aos órgãos da administração direta e indireta da Prefeitura Municipal de São Paulo (PMSP).



### Leitura

In [ ]:
# Preps the query for requesting in Base dos Dados

job_vars = [
    'ano',
    'id_municipio',
    'cep',
    'tipo_estabelecimento',
    'indicador_atividade_ano',
    'quantidade_vinculos_ativos',
    'natureza_juridica',
    'cnae_2',
    ]

query = (
    f"SELECT {', '.join(job_vars)} " 
    "FROM basedosdados.br_me_rais.microdados_estabelecimentos "
    f"WHERE id_municipio IN ({_sql_style_list(tracts.code_muni.unique())}) "
    "AND ano IN (2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023);"
    #f"WHERE id_municipio = '3304557' AND ano = 2014;"
    )

In [ ]:
rais = bd.read_sql(
    query,
    billing_project_id=gcloud_id,
    )

In [ ]:
rais.dtypes

In [ ]:
# Distinction between CNPJ and others
rais = rais.assign(
    tipo_estabelecimento=lambda x: np.where(x.tipo_estabelecimento == '1', 'CNPJ', 'Outros')
    )

In [ ]:
rais.head(3)

In [ ]:
def count_jobs_and_firms(df, grouper='ano'):
    return (
        df
        .groupby(grouper)
        .agg(
            quantidade_estabelecimento=('quantidade_vinculos_ativos', 'size'),
            quantidade_vinculos=('quantidade_vinculos_ativos', 'sum'),
            quantidade_negocios_individuais=('quantidade_vinculos_ativos', negocios_individuais)   
         )
    )


def negocios_individuais(x):
    return x.where(x < 1).replace({0: 1}).sum()

In [ ]:
count_jobs_and_firms(rais)

### Limpeza

A primeira ideia seria utilizar apenas os registros cujo indicador de atividade no ano assumisse o valor de 1, ou seja, apenas estabelecimentos/entidades que exerceras atividade durante o ano de referência. Contudo, análises preliminares (ver tabela abaixo) sugeriram que esse campo, para o ano de 2022 era problemático: 99,5% dos registros para esse ano desapareciam. De fato, esse ano foi o primeiro ano de retorno das atividades logo após a Pandemia de COVID-19, o que pode ter levado a esse resultado. Não obstante, não deixa de ser um resultado estranho, pois no final de 2022 já havia um retorno substancial às atividades. Indo ao encontro dessa situação está o fato de que em 2022 houve uma categoria adicional no indicador de atividade: em vez de apenas 0 ou 1, apareceu o digito 9, cujo significado exato não foi descoberto porque não se localizou nos [layouts de dados](https://www.gov.br/trabalho-e-emprego/pt-br/assuntos/estatisticas-trabalho/microdados-rais-e-caged) disponibilizados pelo Ministério do Trabalho e Emprego.

Ou seja, houve situação atípica em 2022, dado que os valores de inatividade não são plausíveis: por mais que tenha havido um forte impacto na economia, não é razoável que mais da metade da atividade econômica do município tenha sido extinta. As explicações para isso, portanto, fazem parte de uma investigação mais profunda que não faz parte do escopo desta Tese, mas, tudo indica que há um papel importante da categoria 9 no indicador de atividade.

De todo modo, esse fato, levou a uma análise mais aprofundada, com base nas tabelas abaixo. Avaliando todos os anos, com exceção de 2022, percebe-se uma relativa estabilidade no número de estabelecimentos com CNPJ e que empregam pelo menos uma pessoa, assim como uma estabilidade relativa entre os números totais de vínculos empregatícios no município. Isso sugere que há uma flutuação importante entre os totais relativos a locais vinculados a empreendedores individuais. Além da j;a mencionada pandemia, houve ainda [uma mudança metodológica](https://www.gov.br/trabalho-e-emprego/pt-br/assuntos/estatisticas-trabalho/rais/rais-2023/nota-tecnica-rais-2023_11-12-2024.pdf) na RAIS a partir de 2022 e que dificulta a comparação direta entre 2022 e 2023, de um lado, e anos anteriores, de outro.

Nesse contexto, prefere-se trabalhar apenas com estabelecimentos ativos e que empregam pelo menos uma pessoa. Sobretudo, é razoável partir da premissa de que esses locais estão mais diretamente associados com usos do solo não residenciais e que estão mais diretamente ligados ao mecanismo de autômatos celulares que se pretende modelar. Não obstantes, em etapa futura, os dados da RAIS serão comparados com os dados do CNEFE, em etapa que será mais bem detalhada mais à frente.

In [ ]:
# Totais por ano e se executou atividade, por tipo de estab.

count_jobs_and_firms(rais, grouper=['ano', 'indicador_atividade_ano', 'tipo_estabelecimento']).unstack().fillna(0)

In [ ]:
# Estabelecimentos com vínculos de emprego por tipo

(
    rais
    .loc[rais.quantidade_vinculos_ativos > 0]
    .groupby(['ano', 'indicador_atividade_ano', 'tipo_estabelecimento'])
    .size()
    .unstack()
    .fillna(0)
    )

In [ ]:
# Totais de empregos por tipo de estabelecimento

(
    rais
    .groupby(['ano', 'tipo_estabelecimento'])
    .quantidade_vinculos_ativos
    .sum()
    .unstack()
    .fillna(0)
    )

Reiterando, com base na discussão acima, extraímos apenas empresas que empregavam pelo menos 1 pessoa.

In [ ]:
rais = (
    rais
    .loc[rais.quantidade_vinculos_ativos > 0]
)

In [ ]:
count_jobs_and_firms(rais)

Em seguida, dados da RAIS precisam passar por um tratameto para corrigir algumas inconsistências. Para isso, será seguido (com adaptações) o procedimento documentado na linguagem R em https://bit.ly/git_access_op_rais_pre_processing. Esse protocolo foi baseado no raciocínio elaborado em Nadalin (2018)⁕, quem deve ser consultada em caso de maiores detalhes serem necessários.

Primeiro, os empregos de natureza pública são removidos. Essa remoção toma como base os códigos da Tabela de Natureza Jurídica† da CONCLA, do IBGE. Assim, todos os empregos que começam com o dígito 1 serão deletados, bem como o código 2011, que se refere a empresas públicas.

Em seguida, para os empregos das demais naturezas, é feito um tratamento dos outliers. Esse tratamento consiste em identificar setores problemáticos e, para cada um deles, avaliar a distribuição dos vínculos. Caso o estabelecimento de um setor problemático tenha mais vínculos que o percentil 95 da distribuição desse mesmo setor, o total corrigido será igual ao percentil 95 desse mesmo setor. Caso essa condição não seja atendida, o total permanece igual.

Essas anomalias ocorrem porque

    há na iniciativa privada atividades que levam à inconsistência na declaração do total de vínculos ativos que de fato trabalham no endereço informado. É o caso das empresas prestadoras de serviços a outras empresas. São empresas terceirizadas de limpeza, vigilância, manutenção predial, alimentação, entre outras. Há orientação para que seus empregados sejam declarados no quadro de funcionários da empresa terceirizada, mas esses empregados não trabalham na sede dessa empresa, e sim no endereço da empresa contratante dos serviços. Outros casos similares são os de empresas do setor elétrico, de saneamento, de transporte, de transporte aéreo e de limpeza urbana.
    
    Por outro lado, o problema existe quando o estabelecimento declara que todos os seus empregados trabalham em apenas um endereço, mesmo que trabalhem em outros Por outro lado, o problema existe quando o estabelecimento declara que todos os seus empregados trabalham em apenas um endereço, mesmo que trabalhem em outro. (Nadalin, 2018, pp. 4-5)

Cabe destacar que quando excluídas as anomalias, levam-se em consideração apenas os estabelecimentos que empregaram pessoas ao longo do ano em análise. Isso porque há estabelecimentos que funcionaram sem sempregar ninguém, o que enviesaria o teto calcula pelo procedimento acima. Deve-se tem em mente, nesse contexto, que a motivação para corrigir valores inflados e excessivos de alguns setores, conforme elaborado em Nadalin (2018), assim, os valores menores não são razã de grande preocupação.

    ⁕ NADALIN, V. G. Localização de empregos formais e transporte público de massa. Brasília: Ipea, 2018. (Nota Técnica, n. 13). Disponível em <https://bit.ly/2YX6K5O>
    
    † Disponível em <https://bit.ly/concla_nat_jur>

In [ ]:
def flag_public_sector(rais):
    civil_service_jobs = (
        rais.natureza_juridica.str.startswith('1')
        )
    
    state_owned_firms = (rais.natureza_juridica == '2011')
    
    mask = (
        civil_service_jobs | state_owned_firms
        )
    
    return rais.assign(
                    sector=lambda x: np.where(
                                            mask,
                                            'Publico',
                                            'Privado',
                                            )
                    )

In [ ]:
rais = flag_public_sector(rais)

In [ ]:
def remove_public_sector(rais):    
    return rais.loc[rais.sector == 'Privado'].drop(columns='sector')

In [ ]:
rais = remove_public_sector(rais)

In [ ]:
count_jobs_and_firms(rais)

In [ ]:
count_jobs_and_firms(rais).pct_change().mul(100)

Percebe-se aqui um salto entre quando se comparam os anos pré e pós 2022, isto é, há 10% a mais de empregos de 2021 para 2022, o que é atribuído à mudança metodológica na RAIS: a nova metodologia permite maior cobertura e confiança nos registros, de forma que parte desse aumento não se refere exatamente ao mercado de trabalho, mas a uma melhor possibilidade de detecção. No entanto, corrigir para esse fator é algo complexo e que não faz parte do escopo desta tese. Trabalharemos assim com esses número, mas reconhecendo que essa é uma possível limitação. Não entendemos, contudo, que isso vai se caracterizar num problema muito crítico porque as distribuições relativas de emprego e establecimentos no território belorizontino provavelmente estão estáveis — confirmaremos isso adiante.

In [ ]:
def _clean_sector(group):
    problematic_sectors = [
        '35', # ELETRICIDADE, GÁS E OUTRAS UTILIDADES
        '36', # CAPTAÇÃO, TRATAMENTO E DISTRIBUIÇÃO DE ÁGUA
        '38', # COLETA, TRATAMENTO E DISPOSIÇÃO DE RESÍDUOS; RECUPERAÇÃO DE MATERIAIS
        '41', # CONSTRUÇÃO DE EDIFÍCIOS
        '42', # OBRAS DE INFRA-ESTRUTURA
        '43', # SERVIÇOS ESPECIALIZADOS PARA CONSTRUÇÃO
        '49', # TRANSPORTE TERRESTRE
        '51', # TRANSPORTE AÉREO
        '562', # Catering, bufê e outros serviços de comida preparada
        '64', # ATIVIDADES DE SERVIÇOS FINANCEIROS
        '78', # SELEÇÃO, AGENCIAMENTO E LOCAÇÃO DE MÃO-DE-OBRA
        '80', # ATIVIDADES DE VIGILÂNCIA, SEGURANÇA E INVESTIGAÇÃO
        '81', # SERVIÇOS PARA EDIFÍCIOS E ATIVIDADES PAISAGÍSTICAS
        '82', # SERVIÇOS DE ESCRITÓRIO, DE APOIO ADMINISTRATIVO E OUTROS SERVIÇOS PRESTADOS A EMPRESAS
        '84', # ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL
        ]
    sector_mask = group.cnae_2.str.startswith(tuple(problematic_sectors))
        
    ceiling = (
        group
        .loc[sector_mask, 'quantidade_vinculos_ativos']
        .pipe(
            lambda x: x[x > 0]
            )
        .quantile(.95)
        )
        
    job_mask = group.quantidade_vinculos_ativos > ceiling
    
    mask = sector_mask & job_mask
    
    group['quantidade_vinculos_ativos'] = np.where(
                                            mask,
                                            ceiling,
                                            group.quantidade_vinculos_ativos,
                                            )
    return group
    

def clean_job_outliers(rais):
    return (
        rais
        .groupby(['ano', 'cnae_2'])
        .apply(_clean_sector)
        .astype({'quantidade_vinculos_ativos': float})
        .round({'quantidade_vinculos_ativos': 0})
        .astype({'quantidade_vinculos_ativos': int})
        )

In [ ]:
rais = clean_job_outliers(rais)

In [ ]:
rais.head(2)

In [ ]:
count_jobs_and_firms(rais)

### Filtros: Setores de Interesse

Em seguida, são removidos os setores econômicos que se entende que não são muito afetados pelas dinâmicas urbanas que o modelo se propõe a simular. Agricultura é um deles, sobretudo porque não é algo representativo na cidade de Belo Horizonte. De forma similar, as contagens de usos industriais não serão levados em conta, eles são pouco representativos na cidade e são majoritariamente definidos por questão extra-urbanas: inserção da indústria na economia regional e, até mesmo, nacional. Esse tipo de relação é de difícil modelagem em nosso contexto, dado que a área de estudo se limia ao iterior do município de Belo Horizonte apenas. Ou seja, modelar a relocação da indústria envolve medir a acessibilidade para além do território municipal e questões logísticas, incluindo até mesmo a Região Metropolitana como um todo. Nesse contexto, opta-se por tratar a indústria como um uso estático e que irá influenciar os demais. De fato, apesar do seu notável poder estruturador, a indústria apresenta relativa independência das questões urbanas mais locais (Villaça, 2017, p. 140).

Ainda, atividades de ELETRICIDADE E GÁS; ÁGUA, ESGOTO, ATIVIDADES DE GESTÃO DE RESÍDUOS E DESCONTAMINAÇÃO; e TRANSPORTE, ARMAZENAGEM E CORREIO também não serão contabilizadas. Parte-se da premissa de que elas estão mais associadas a lógicas próprias e não são exatamente comerciais, no sentido de fornecer serviço ou atendimento ao público, tal como são as demais categorias mantidas para análise. Ente-se que essas atividades estão pautadas por questões também de caráter mais logísticos, referentes às necessidades das redes e infraestruturas próprias que atendem. No caso de atividades de construção, parte-se da premissa que elas são uma espécie de atividade transitória. De fato, ela é responsável, ao cabo, por efetuar a transição entre usos do solo, mas talvez fossem mais pertinentes de serem incorporadas ao modelo se houvesse modelagem explícita a um uso do solo caracterizado como construção, o que não é o caso.

Assim, são mantidas as demais categorias presentes na RAIS, que são baseadas no [CONCLA](https://cnae.ibge.gov.br/?view=estrutura).

In [ ]:
cnae_sections = [
    (0, 3),
    (5, 9),
    (10, 33),
    (35, 35.1), # Apenas um artificio para gerar um intervalo
    (36, 39),   # pois ha so uma subdivisao. Isso se repete abaixo
    (41, 43),   # algumas vezes
    (45, 47),
    (49, 53),
    (55, 56),
    (58, 63),
    (64, 66),
    (68, 68.1),
    (69, 75),
    (77, 82),
    (84, 84.1),
    (85, 85.1),
    (86, 88),
    (90, 93),
    (94, 96),
    (97, 97.1),
    (99, 99.1),
]

cnae_sections = pd.IntervalIndex.from_tuples(cnae_sections, closed='both')

binned_cnae = pd.cut(
    rais.cnae_2.str.extract(r'(\d\d)').astype(int).squeeze(),
    bins=cnae_sections,
)

labeled_cnae = binned_cnae.astype(str).replace({
    str(cat): letter
    for cat, letter
    in zip(
        cnae_sections,
        ascii_uppercase[:len(cnae_sections)]
    )
})

rais['grupo_cnae'] = labeled_cnae

In [ ]:
rais.head()

Antes da remoção dos setores selecionados acima, contudo, a seguir são demonstrados graficamente os totais de empregos por grupo do CNAE.

In [ ]:
sns.set_style("whitegrid")
sns.set_context("paper")

ax = sns.barplot(
    data=rais.groupby(['grupo_cnae', 'ano'], as_index=False).size().astype({'ano': 'str'}),
    x='grupo_cnae',
    y='size',
    hue='ano',
    linewidth=0,
    palette=sns.color_palette("crest", n_colors=len(range(2014, 2024))),
    )

plt.setp(ax.get_legend().get_texts(), fontsize='12') # for legend text
plt.setp(ax.get_legend().get_title(), fontsize='16') # for legend title

ax.set_xlabel("Grupo CNAE", fontsize=14)
ax.set_ylabel("Quantidade de Estabelecimentos", fontsize=14)
ax.tick_params(labelsize='large')

sns.despine()

O gráfico mostra que a remoção conforme discutido acima não gera distorções muito significativas, pois os setores removidos são relativamente pouco representativos em Belo Horizonte. Construção, de fato, tem uma quantidade não desprezível, mas, pelos motivos discutidos anteriormente, será removida mesmo assim. De qualquer forma, o grupo G — COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E MOTOCICLETAS — é de longe o mais representativo.

In [ ]:
cnae_filter = (
    [f"{c:02}" for c in range(45)]
    + [f"{c:02}" for c in range(49, 54)]
    + ['97', '99']
)

rais = rais.loc[~rais.cnae_2.str.startswith(tuple(cnae_filter))]

In [ ]:
count_jobs_and_firms(rais)

In [ ]:
rais.head()

### Totalização

Foi cogitado separar os setores emprimário e secundário, mas como visto no artigo, ambos tem localizações muito similares e, ao que tudo indica, seus padrões espaciais são gerados por processos similares, portanto, diferentemente de Porta et al. (2012), não faremos aqui essa distinção. Além disso, agora que outliers foram tratados e foram removidos setores que não são de interesse, será feita uma totalização da quantidade de empregos por CEP e por ano.

Agora serão totalizados os empregos para cada CEP, mantendo as classes de emprego acima.

In [ ]:
rais = (
    rais.groupby(
            ['ano', 'id_municipio', 'cep'],
            as_index=False,
            )
        .agg({
            'quantidade_vinculos_ativos': [np.sum, 'size'],
            })
    )

rais.columns = ['_'.join(c).rstrip('_') for c in rais.columns.values]

In [ ]:
rais = rais.rename(
    columns={
        'quantidade_vinculos_ativos_sum': 'quantidade_vinculos_ativos',
        'quantidade_vinculos_ativos_size': 'quantidade_estabelecimentos',
    }
)

In [ ]:
rais.head()

In [ ]:
rais.groupby('ano').agg({'quantidade_vinculos_ativos': 'sum', 'quantidade_estabelecimentos': 'sum' })

# Operações Espaciais

Esta seção objetiva imputar os atributos territoriais trabalhados anteriormente nas malhas hexagonais com as resoluções pré-definidas no item 1.1. Com efeito, serão executados cinco grandes grupos de operações espaciais:
1. Primeiro, geolocalizam-se os CEPs da RAIS a partir dos CEPs.
2. Em seguida, dados de cada setor são atribuídos às respectivas geometrias e, ainda, é corrigida uma anomalia identificada na definição do polígono de um dos setores.
3. Depois, as informações (a) de contagem populacional e (b) de rendimentos são transferidas para os hexágonos por meio de procedimentos de interpolação espacial.
4. Logo após, junções (spatial joins) são aplicada aos hexágonos de base, de um lado, e aos postos de trabalho geolocalizados, de outro.
5. Por fim, são encontrados os centroides de cada hexagonos, a partir de uma ponderação que deixa esse ponto mais próximo do centro de massa da população, o que em muitos casos provocará um afastamento em relação ao centro geométrico.

## Geolocalização da RAIS

Os dados da RAIS não estão geolocalizados a priori: eles contêm informações a respeito do CEP de cada estabelecimento comercial, de maneira que uma correta localização no território exige associar o CEP à uma coordenada geográfica que bom o represente.

De início, aproveitam-se as bases de endereços presentes nas três cidades em estudo, que permitem localizar a maiori dos CEPs. Para os casos remanescentes, utiliza-se a base de CEPs que está hospedada na Base dos Dados e, caso ainda sobre algum CEP não geolocalizado, parte-se para o uso de APIs. Há duas as quais aqui se dá destaque: o serviço de geocodificação do Google⁕ e o projeto Brasil API†. Ambas tomam o CEP como ponto de partida e retornam o nome dos logradouros e, ainda, a latitude e a longitude (no sistema de coordenadas EPSG:4326 — WGS 84).


Enfim, é feita uma avaliação quanto à necessidade de correção há erros e anomalias particulares de cada cidade.

⁕ ver https://developers.google.com/maps/documentation/javascript/geocoding?hl=pt-br

† ver https://brasilapi.com.br/docs#tag/CEP-V2 ou https://github.com/BrasilAPI/BrasilAPI. Especificamente, o código fonte de interesse está em https://github.com/BrasilAPI/cep-promise

### Padronização dos CEPs

In [ ]:
rais.cep.map(len).describe()

Em alguns casos, os CEPs começam com um zero à esquerda, o que pode fazer com que o dígito à esquerda seja perdido nas manipulações computacionais (isso é um problema sobretudo em São Paulo). Para garantir que não haverá problemas (mesmo que a celula acima indique que nao), força-se os CEPs a sempre terem 8 dígitos: quando há 7, acrescenta-se um zero à esquerda.

In [ ]:
rais['cep'] = rais.cep.map('{:>08}'.format)

### Enderecos

In [ ]:
# Dados de endereços e respectivos ceps
inpath = db / 'beaga/enderecos.zip'

addresses = (
    gpd
    .read_file(inpath)
    .rename(
        columns=lambda x: x.casefold()
    ))

addresses.head(3)

In [ ]:
addresses = (
    addresses
    .dropna(subset='cep')
    .astype({'cep': 'int'})
    .astype({'cep': 'str'})
    )

In [ ]:
# Espalhando empregos pelos endereços de um mesmo cep aleatoriamente
def _random_spread(df):
    def distribute(total, candidates, weights=None):
        if weights is None or weights.sum() == 0:
            weights = np.ones(len(candidates))  # Equal distribution
        weights = weights / weights.sum()  # Normalize weights
        
        allocations = np.random.multinomial(total, weights)
        
        return pd.Series(allocations, index=candidates)

    # Distribute jobs
    job_total = df['quantidade_vinculos_ativos'].iloc[0]
    df['quantidade_vinculos_ativos'] = distribute(job_total, df.index)

    # Distribute shops weighted by jobs
    shop_total = df['quantidade_estabelecimentos'].iloc[0]
    weights = df['quantidade_vinculos_ativos'].to_numpy()
    df['quantidade_estabelecimentos'] = distribute(shop_total, df.index, weights)

    return df


def distribute_rais(rais, ceps):
    rng = np.random.default_rng(42)

    basket = []
    for idx, data in rais.groupby('ano'):
        assert data.cep.is_unique, "CEPs not unique"
        
        df = (
            ceps
            .merge(data, on='cep')
            .pipe(lambda x: x.loc[x.geometry.notnull()])
            .groupby('cep', group_keys=False)
            .apply(_random_spread)
            )

        basket.append(df)

    return pd.concat(basket)

        

In [ ]:
rais = distribute_rais(
    rais,
    addresses[['cep', 'geometry']],
    )

In [ ]:
rais.head(3)

In [ ]:
rais.groupby('ano').agg({'quantidade_vinculos_ativos': 'sum', 'quantidade_estabelecimentos': 'sum' })

## Imputacao: Dados Censitários -> Malhas Censitarias

Esta parte faz a junçao dos dados dos setores censitários com as respectivas geometrias dos setores. Após esse processo, será feita a interpolação espacial dos dados para os Hexágonos H3.

In [ ]:
# União entre malha de setores e respectivos dados
tracts_with_data = (
    tracts
    .merge(
        tract_data,
        left_on='code_tract',
        right_on='id_setor_censitario',
        how='left',
    )
    .merge(
        ibeu,
        left_on='id_setor_censitario',
        right_index=True,
        how='left',
    )
    .reindex(
        columns=[
            'id_setor_censitario',
            'code_muni',
            'name_muni',
            'habitantes',
            'domicilios',
            'rendimento_medio',
            'ibeu',
            'geometry',
        ]
    )
)

tracts_with_data.head()

## Interpolações: Setor Censitário → Hexágonos

In [ ]:
hexes.head(2)

In [ ]:
basket = []
for aperture, data in hexes.groupby(level='aperture'):
    hex_with_data = (
        area_interpolate(
            source_df=tracts_with_data.assign(
                geometry=lambda x: x.buffer(0)
                ),
            target_df=data,
            extensive_variables=['habitantes', 'domicilios'],
            intensive_variables=['rendimento_medio', 'ibeu']
            )
        .drop(columns='geometry')
        .set_index(data.index)
        .pipe(
            lambda df: pd.merge(
                hexes,
                df,
                left_index=True,
                right_index=True,
                )
            )
        )

    basket.append(hex_with_data)

hex_with_data = pd.concat(basket)

In [ ]:
hex_with_data.head(2)

In [ ]:
hex_with_data.filter(like='hab').groupby(level=['name_muni', 'aperture']).sum()

In [ ]:
hex_with_data.reset_index().hex_id.is_unique

## Junções Espaciais: Rais → H3

In [ ]:
hexes = (
    hexes
    .reset_index()
    .sjoin(
        rais.loc[rais.filter(like='quant').sum(1) > 0],
        how='left',
    )
    .drop(columns=['index_right'])
    .dissolve(
        by=['name_muni', 'code_muni', 'aperture', 'hex_id', 'ano'],
        aggfunc='sum',
        )
    .sort_index()
)

In [ ]:
hexes.head(3)

In [ ]:
hexes.groupby(['ano', 'aperture']).agg({'quantidade_vinculos_ativos': 'sum', 'quantidade_estabelecimentos': 'sum'})

In [ ]:
basket = []

for (year, res), data in hexes.groupby(['ano', 'aperture']):
    df = (
        hex_with_data
        .xs(level='aperture', key=res)
        .reset_index()
        .merge(
            data.reset_index().drop(columns=['geometry', 'name_muni', 'code_muni']),
            on='hex_id',
            how='left'
            )
        .assign(
            aperture=res,
            ano=year,
            )
        .set_index(data.index.names)
        )
    basket.append(df)
        

hexes = pd.concat(basket)

In [ ]:
(
    hexes
    .groupby(['ano', 'aperture'])
    .agg({
        'quantidade_vinculos_ativos': 'sum',
        'quantidade_estabelecimentos': 'sum',
        'habitantes':'sum'
        })
    )

# Epílogo: Censo 2022

A base dos dados parecia estar com erro na tabela necessaria, portanto fui da forma manual com base nos dados [daqui](https://www.ibge.gov.br/estatisticas/sociais/populacao/22827-censo-demografico-2022.html?edicao=41852&t=resultados).

In [ ]:
inpath = db / 'census/malha_setores_2022.gpkg'

tracts = (
    gpd
    .read_file(inpath)
    .pipe(
        lambda df: df.loc[df.NM_MUN == 'Belo Horizonte']
        )
    .reindex(
        columns=['CD_SETOR', 'v0001', 'v0002', 'geometry']
        )
    .rename(columns={'v0001': 'habitantes', 'v0002': 'domicilios'})
    )

In [ ]:
tracts.head(3)

In [ ]:
def clear_future_years(db):
    return (
        db
        .reset_index()
        .assign(
            habitantes=lambda x: np.where(x.ano != 2014, np.nan, x.habitantes),
            domicilios=lambda x: np.where(x.ano != 2014, np.nan, x.domicilios),
            )
        .set_index(db.index.names)
        )


hexes = clear_future_years(hexes)

In [ ]:
basket = []
for (aperture, year), data in hexes.groupby(level=['aperture', 'ano']):
    if year == 2023:
        hex_with_data = (
            area_interpolate(
                source_df=tracts.to_crs(31983).assign(
                    geometry=lambda x: x.buffer(0)
                    ),
                target_df=data,
                extensive_variables=['habitantes', 'domicilios'],
                )
            .drop(columns='geometry')
            .set_index(data.index)

            )
        basket.append(hex_with_data)

hex_with_data = pd.concat(basket)

In [ ]:
hex_with_data.head()

In [ ]:
hex_with_data.groupby('aperture').agg({'habitantes': 'sum', 'domicilios': 'sum'})

In [ ]:
hexes.update(hex_with_data)

In [ ]:
def interpolate_middle_years(db, col):
    return (
        db
        .groupby('hex_id')
        [col]
        .transform(
            lambda t: t.interpolate()
            )
        )

hexes = (
    hexes
    .sort_values(['ano', 'hex_id'])
    .assign(
        habitantes=lambda db: interpolate_middle_years(db, 'habitantes'),
        domicilios=lambda db: interpolate_middle_years(db, 'domicilios')
        )
    )

In [ ]:
(
    hexes
    .groupby(['ano', 'aperture'])
    .agg({
        'quantidade_vinculos_ativos': 'sum',
        'quantidade_estabelecimentos': 'sum',
        'habitantes':'sum',
        'domicilios': 'sum',
        })
    )

# Backup

In [ ]:
outpath = out_folder / 'hexagonos.gpkg'

hexes.reset_index().to_file(outpath, layer='socioeconomico')